# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234616434077                   -0.50    8.0
  2   -7.250365272282       -1.80       -1.40    1.0
  3   -7.251306186717       -3.03       -2.15    4.0
  4   -7.251258290837   +   -4.32       -2.19    3.0
  5   -7.251329115934       -4.15       -2.67    2.0
  6   -7.251337987482       -5.05       -3.16    2.0
  7   -7.251338759277       -6.11       -3.90    1.0
  8   -7.251338793896       -7.46       -4.18    3.0
  9   -7.251338798087       -8.38       -4.63    2.0
 10   -7.251338798662       -9.24       -5.23    3.0
 11   -7.251338798701      -10.41       -5.76    3.0
 12   -7.251338798704      -11.52       -6.17    2.0
 13   -7.251338798705      -12.37       -7.04    3.0
 14   -7.251338798705      -14.01       -7.33    2.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.048691786670096465
[ Info: Arnoldi iteration step 2: normres = 0.6221230917635626
[ Info: Arnoldi iteration step 3: normres = 0.6395374918051614
[ Info: Arnoldi iteration step 4: normres = 0.2951538265080885
[ Info: Arnoldi iteration step 5: normres = 0.3349881798055827
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (9.27e-03, 7.17e-02, 2.68e-01, 1.85e-01, 2.87e-02)
[ Info: Arnoldi iteration step 6: normres = 0.47297146867256257
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (3.62e-03, 1.57e-01, 4.22e-01, 1.18e-01, 6.91e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08733922336939556
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.77e-04, 2.91e-02, 1.99e-02, 4.28e-02, 5.63e-02)
[ Info: Arnoldi iteration step 8: normres = 0.1276390855876241
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (9.71e-